In [1]:
import matplotlib.pyplot as plt
import pandas as pd
from math import log
from re import compile
from urllib.parse import urlparse
from socket import gethostbyname
from requests import get
from string import ascii_lowercase
from numpy import array
import time
from datetime import datetime
from re import compile
from json import dump, loads
from time import sleep

# feature extract our URL :

In [3]:

def get_entropy(URL):
    URL = URL.lower()
    probs = [URL.count(c) / len(URL) for c in set(URL)]
    entropy = -sum([p * log(p) / log(2.0) for p in probs])
    return entropy

def url_has_port_in_string(URL_parsed):
    try :
        has_port = URL_parsed.netloc.split(':')
        return 1 if len(has_port) > 1 and has_port[-1].isdigit() else 0
    except :
        return 0

def number_of_digits(URL):
    try :
        digits = [i for i in URL if i.isdigit()]
        return len(digits)
    except :
        return 0

def number_of_parameters(URL_parsed):
    try :
        params = URL_parsed.query
        return 0 if params == '' else len(params.split('&'))
    except :
        return 0


def number_of_fragments(URL_parsed):
    try :
        frags = URL_parsed.fragment
        return frags.count('#') if frags != '' else 0
    except :
        return 0


def is_encoded(URL):
    try:
        return 1 if '%' in URL else 0
    except :
        return 0



def number_of_dots(URL):
    try :
        dots = [i for i in URL if i == '.'] ## ??
        return len(dots)
    except :
        return 0

def has_client_in_string(URL):
    try :
        return 1 if 'client' in URL.lower() else 0
    except :
        return 0

def has_admin_in_string(URL):
    try :
        return 1 if 'admin' in URL.lower() else 0
    except :
        return 0

def has_server_in_string(URL):
    try :
       return 1 if 'server' in URL.lower() else 0
    except :
        return 0

def has_login_in_string(URL):
    try :
         return 1 if 'login' in URL.lower() else 0
    except :
        return 0

def get_tld(URL_parsed): # https://miro.medium.com/v2/resize:fi
                                             #----
    try :
        if not url_host_is_ip(URL_parsed) :
            tld = URL_parsed.netloc.split('.')[-1].split(':')[0]
            return tld if len(tld)<10 else ''
        else: return ''
    except :
        return ''
    
def url_host_is_ip(URL_parsed):
    try :
        host = URL_parsed.netloc
        pattern = compile("^\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3}$")
        match = pattern.match(host)
        return 1 if match is not None else 0
    except :
        return 0

def CharacterContinuityRate(URL_parsed):
    try :

        if not url_host_is_ip(URL_parsed):
            Domain = URL_parsed.geturl()
            counter = dict()
            counter['digit'] = 0
            counter['alpha'] = 0
            counter['symbol'] = 0
            counter_digit__ = 0
            counter_alpha__ = 0
            counter_symbol__ = 0
            was_alpha = False
            was_digit = False
            was_symbol = False
            for c in Domain :
                if c.isdigit() :
                    if was_digit :
                        counter_digit__ +=1
                    else : 
                        was_alpha, was_symbol, was_digit = False ,False, True
                        counter_digit__  = 1
                    counter['digit']= max(counter['digit'], counter_digit__)


                elif c.isalpha() :
                    if was_alpha :
                        counter_alpha__ +=1
                    else : 
                        was_digit, was_symbol, was_alpha = False ,False, True
                        counter_alpha__  = 1
                    counter['alpha']= max(counter['alpha'], counter_alpha__)
                else  :                  
                    if was_symbol : 
                        counter_symbol__ +=1
                    else : 
                        was_digit, was_alpha, was_symbol = False ,False, True
                        counter_symbol__  = 1
                    counter['symbol']= max(counter['symbol'], counter_symbol__)
            return (counter['alpha']+ counter['digit']+ counter['symbol'])/len(Domain)
        else :
            return 4/15
    except :
        return 0
    

def domain_token_count(URL):
    try:
        token_count= URL.count('/')
        return token_count
    except :
        return 0

def Path_Domain_ratio(URL_parsed):
    try :
        path_ = len(URL_parsed.path)
        domain_ = len(URL_parsed.netloc)
        return path_ / domain_
    except :
        return 0


def Path_URL_ratio(URL_parsed):
        try :
            path_ = len(URL_parsed.path)
            URL_ = len(URL_parsed.geturl())
            return path_ / URL_
        except :
            return 0


def  Query_Digit_Count(URL_parsed):# Number of digits in the query part of the URL.
    try :
        digits = [i for i in URL_parsed.query if i.isdigit()]
        return len(digits)
    except :
        return 0

def Longest_Path_token(URL_parsed):
    try :
        Path_ = URL_parsed.path
        lengths_ = list()
        counter = 0
        for c in Path_ :
            if c == '/': 
                lengths_.append(counter)
                counter = 0
            else: counter += 1

        return max(lengths_)
    except :
        return 0

def Symbol_Count_Domain(URL):  #in "://.:/?=,;()]+"
    try :
        counter = 0  
        for c in URL:
            if c in "://.:/?=,;()]+" : counter += 1
        return counter
    except :
        return 0




In [4]:
def __get_ip(URL_parsed):
    try:
        ip = URL_parsed.netloc if url_host_is_ip(URL_parsed) else gethostbyname(URL_parsed.netloc)
        return ip
    except:
        return None
    
	
def URL_scheme(URL_parse):
    try :
        scheme = URL_parse.scheme
        return scheme
    except :
        return ''



In [5]:
import validators.url as url


def URLParse_with_exeption(URL):
    try :
        url_parse = urlparse(URL)
        return url_parse
    except ValueError :
        print(URL)
        return None
def validate_url(URL):
    if not url(URL):
        URL = "NoScheme://" + URL
    return URL





In [1]:
import pandas as pd
import numpy as np
# import category_encoders as ce
URL = input('URL...................\n')
One_URL_df = pd.DataFrame( data=[ URL] , columns=[ 'url'])
One_URL_df.head()

my_encoder = joblib.load('Encoder_BDE.gz')
my_scaler = joblib.load('StandardScaler.gz')
my_PCA = joblib.load('PCA.gz')
my_LSTM_model = joblib.load('LSTM_model.gz')


############################ lecixal features #############################################

One_URL_df["entropy"] = One_URL_df["url"].apply(get_entropy)
One_URL_df["url_parsed"] = One_URL_df["url"].apply(validate_url).apply(URLParse_with_exeption)
One_URL_df["has_a_port"] = One_URL_df["url_parsed"].apply(url_has_port_in_string)
One_URL_df["number_of_digits"] = One_URL_df["url"].apply(number_of_digits)
One_URL_df["number_of_parameters"] = One_URL_df["url_parsed"].apply(number_of_parameters)
One_URL_df["number_of_fragments"] = One_URL_df["url_parsed"].apply(number_of_fragments)
One_URL_df['URL_host_is_IP'] = One_URL_df['url_parsed'].apply(url_host_is_ip)
One_URL_df["is_encoded"] = One_URL_df["url"].apply(is_encoded)
One_URL_df["has_client"] = One_URL_df["url"].apply(has_client_in_string)
One_URL_df["has_admin"] = One_URL_df["url"].apply(has_admin_in_string)
One_URL_df["has_server"] = One_URL_df["url"].apply(has_server_in_string)
One_URL_df["has_login"] = One_URL_df["url"].apply(has_login_in_string)
One_URL_df["number_of_dots"] = One_URL_df["url"].apply(number_of_dots)
One_URL_df["CCR"] = One_URL_df["url_parsed"].apply(CharacterContinuityRate)
One_URL_df["Symbol_Count_Domain"] = One_URL_df["url"].apply(Symbol_Count_Domain)
One_URL_df["Longest_Path_token"] = One_URL_df["url_parsed"].apply(Longest_Path_token)
One_URL_df["Query_Digit_Count"] = One_URL_df["url_parsed"].apply(Query_Digit_Count)
One_URL_df["Path_URL_ratio"] = One_URL_df["url_parsed"].apply(Path_URL_ratio)
One_URL_df["Path_Domain_ratio"] = One_URL_df["url_parsed"].apply(Path_Domain_ratio)
One_URL_df["domain_token_count"] = One_URL_df["url"].apply(domain_token_count)

############################ Host_Based features #############################################

One_URL_df["tld"] = One_URL_df["url_parsed"].apply(get_tld)
One_URL_df["URL_scheme"] = One_URL_df["url_parsed"].apply(URL_scheme)

###### encoding #######

One_URL_df = One_URL_df.drop(columns=('url_parsed'),)

obj_df = One_URL_df.select_dtypes(include=['object'])
df_bd = my_encoder.transform(One_URL_df)
One_URL_df_bd = df_bd.iloc[:,1:]

###### scaling #######

# from sklearn.preprocessing import StandardScaler, MinMaxScaler

numeric_df = One_URL_df_bd.select_dtypes(include=['int16', 'int32', 'int64', 'float16', 'float32', 'float64'])
numeric_values = numeric_df.values
numeric_values = my_scaler.transform(numeric_df.to_numpy())
scaled_numeric_df = pd.DataFrame(data = numeric_values
             , columns = numeric_df.columns )
###### PCA ############


Principal_C = my_PCA.transform(scaled_numeric_df)

k = my_PCA.n_components_

principalDf = pd.DataFrame(data = Principal_C
             , columns = ['Principal_C'+str(i) for i in range(1,k+1)])
principalDf.head()

###### the last result ########
result = One_URL_df_bd.drop(numeric_df.columns, axis=1)
New_on_URL_DF =  result.join( principalDf,) 
New_on_URL_DF.head()


NameError: name 'joblib' is not defined

In [ ]:
import tensorflow as tf
X = New_on_URL_DF.iloc[:,[i for i in range(1,35)] ].values
X = X.reshape(34,1)
predicted_class=my_LSTM_model.predict(tf.expand_dims(X,0)).argmax()
print(predicted_class)